In [1]:
import random

In [2]:
#Initialise 
P10 = [7, 3, 6, 1, 10, 4, 2, 5, 8, 9]
P8 = [1, 4, 2, 6, 7, 3, 10, 8]
IP = [3, 8, 1, 5, 4, 2, 6, 7]
IP_INV = [3, 6, 1, 5, 4, 7, 8, 2]
EP = [2, 4, 3, 1, 2, 4, 3, 1]
S0 = [
    [1, 0, 3, 2],
    [3, 2, 1, 0],
    [0, 2, 1, 3],
    [3, 1, 3, 2]
]
S1 = [
    [0, 1, 2, 3],
    [2, 0, 1, 3],
    [3, 0, 1, 0],
    [2, 1, 0, 3]
]

In [3]:
#Conversion
def bin_to_dec(x):
  return int(x, 2)
def dec_to_bin(x):
  return bin(x).replace("0b","")

#Left Shift
def left_circular_shift(x, shifts=1):
  shifts = shifts % len(x)
  return x[shifts:] + x[:shifts]

#Permute based on Initialization
def permutate(key, perm):
  ret = ""
  for k in perm:
    ret += key[k-1]
  return ret

#Split String
def split_str(key):
  half = len(key)//2
  key1 = key[:half]
  key2 = key[half:]
  return key1, key2

#Xor Bianry
def xor(a, b):
  ret = ""
  for i in range(len(a)):
    if a[i] == b[i]: ret += "0"
    else: ret += "1"
  return ret

### Procedure

In [4]:
def gen_subkeys(key):
  n_key = permutate(key, P10)

  left_key, right_key = split_str(n_key)

  left_key = left_circular_shift(left_key, 1)
  right_key = left_circular_shift(right_key, 1)

  k1 = permutate(left_key + right_key, P8)

  left_key = left_circular_shift(left_key, 2)
  right_key = left_circular_shift(right_key, 2)

  k2 = permutate(left_key + right_key, P8)

  return k1, k2

In [5]:
#S-Box
def s_box(text, s):
  r = text[0] + text[3]
  c = text[1] + text[2]

  r = bin_to_dec(r)
  c = bin_to_dec(c)
  out = s[r][c]
  out = dec_to_bin(out)
  while len(out) < 2:
    out = "0" + out
  return out


#Function
def function(left, right, subkey):
  text = right
  text = permutate(text, EP)
  text = xor(text, subkey)
  text_left, text_right = split_str(text)
  text = s_box(text_left, S0) + s_box(text_right, S1)
  text = xor(text, left)
  return text, right

In [6]:
def encryption(plaintext, key):
  k1, k2 = gen_subkeys(key)

  ciphertext = permutate(plaintext, IP)

  left, right = split_str(ciphertext)
  left, right = function(left, right, k1)

  left, right = right, left

  left, right = function(left, right, k2)

  ciphertext = permutate(left + right, IP_INV)

  return ciphertext

In [7]:
def decryption(ciphertext, key):
  k1, k2 = gen_subkeys(key)

  plaintext = permutate(ciphertext, IP)

  left, right = split_str(plaintext)
  left, right = function(left, right, k2)

  left, right = right, left

  left, right = function(left, right, k1)

  plaintext = permutate(left + right, IP_INV)

  return plaintext

In [8]:
key = input("Enter 10 bit key: ")
plaintext = input("Enter 8 bit Plain Text: ")

c = encryption(plaintext, key)
print("\nEncrypted Test is : ", c)
p = decryption(c, key)
print("\nDecrypted Text is : ", p)

assert(plaintext == p)

Enter 10 bit key: 1010001011
Enter 8 bit Plain Text: 10001010

Encrypted Test is :  00001110

Decrypted Text is :  10001010
